In [5]:
import torch
from typing import Tuple


# Q1 Gradient-based Factorisation

In [6]:
def sgd_factorise(A: torch.Tensor, rank: int, num_epochs=1000, lr=0.01) -> Tuple[torch.Tensor, torch.Tensor]:
    m, n = A.shape

    U = torch.rand((m, rank))
    V = torch.rand((n, rank))

    for i in range(num_epochs):
        for r in range(m):
            for c in range(n):
                e = A[r, c] - U[r] @ V[c].T
                U[r] = U[r] + lr * e * V[c]
                V[c] = V[c] + lr * e * U[r]

    return U, V


# Q2 Compute reconstruction error

In [7]:
A = torch.tensor([
    [0.3374, 0.6005, 0.1735],
    [3.3359, 0.0492, 1.8374],
    [2.9407, 0.5301, 2.2620]
])

# reconstruction
U, V = sgd_factorise(A, 2)

A_G = U @ V.T

print('U,', U)
print('V', V)
print('A\': ', A_G)

U, tensor([[ 0.5660, -0.2228],
        [ 0.6841,  1.4940],
        [ 1.2340,  1.0253]])
V tensor([[ 1.0608,  1.6883],
        [ 0.7862, -0.3655],
        [ 0.9734,  0.8787]])
A':  tensor([[ 0.2243,  0.5264,  0.3552],
        [ 3.2481, -0.0083,  1.9787],
        [ 3.0401,  0.5954,  2.1021]])


In [8]:
# loss 
print('loss:', torch.trace((A - A_G) @ (A - A_G).T))

# loss is given by the first singular value that we didn’t use
# i.e. 0.3492**2 == 0.1219
loss = torch.nn.MSELoss(reduction='sum')
print('loss:', loss(A, A_G))



loss: tensor(0.1220)
loss: tensor(0.1220)


# Q3 Truncated SVD

In [5]:
U, S, V = torch.svd(A)

U, S, V 


(tensor([[-0.0801, -0.7448,  0.6625],
         [-0.7103,  0.5090,  0.4863],
         [-0.6994, -0.4316, -0.5697]]),
 tensor([5.3339, 0.6959, 0.3492]),
 tensor([[-0.8349,  0.2548,  0.4879],
         [-0.0851, -0.9355,  0.3430],
         [-0.5439, -0.2448, -0.8027]]))

In [6]:
# remove the last singular
S[-1] = 0

# reconstruction
A_hat = U @ torch.diag(S) @ V.T
print('A_hat', A_hat)

# loss 
print('loss:', torch.trace((A - A_hat) @ (A - A_hat).T))

# loss is given by the first singular value that we didn’t use
# i.e. 0.3492**2 == 0.1219
loss = torch.nn.MSELoss(reduction='sum')
print('loss:', loss(A, A_hat))


A_hat tensor([[ 0.2245,  0.5212,  0.3592],
        [ 3.2530, -0.0090,  1.9737],
        [ 3.0378,  0.5983,  2.1023]])
loss: tensor(0.1219)
loss: tensor(0.1219)


# Q4 Masked factorisation 

In [7]:
def sgd_factorise_masked(A: torch.Tensor, M: torch.Tensor, rank: int, num_epochs=1000, lr=0.01) -> Tuple[torch.Tensor, torch.Tensor]:
    m, n = A.shape

    U = torch.rand((m, rank))
    V = torch.rand((n, rank))

    for i in range(num_epochs):
        for r in range(m):
            for c in range(n):
                if M[r, c]:
                    e = A[r, c] - U[r] @ V[c].T
                    U[r] = U[r] + lr * e * V[c]
                    V[c] = V[c] + lr * e * U[r]

    return U, V


# Q5 Reconstruct a matrix

In [8]:
M = torch.tensor([
    [1, 1, 1],
    [0, 1, 1],
    [1, 0, 1]
])

U, V = sgd_factorise_masked(A, M, 2)

A_Complete = U @ V.T

A_Complete


tensor([[0.3357, 0.6007, 0.1757],
        [2.3070, 0.0491, 1.8379],
        [2.9410, 0.5899, 2.2615]])